In [4]:
import os 
os.environ['IMAGEIO_FFMPEG_EXE'] = '/Users/shahzebnaveed/data/noise/ffmpeg'

from moviepy.editor import VideoFileClip
import librosa
import numpy as np

output_path = 'cleaned_video.mov'

def remove_noise(video_path, output_path="output.mov", sr=44100):
  """
  Attempts to remove background noise from a .MOV video file.

  Args:
    video_path: Path to the input video file.
    output_path: Path to save the output video with noise removed (default: output.mov).
    sr: Sample rate of the audio (default: 44100 Hz).
  """
  clip = VideoFileClip(video_path)
  audio = clip.audio.to_soundarray()

  # Apply spectral noise reduction using librosa
  y, sr = librosa.load(video_path, sr=sr)
  S = librosa.stft(y)  # Short-time Fourier Transform
  S_db = librosa.power_to_db(S, ref=np.max(abs(S)))  # Convert to decibel scale

  # Estimate noise using spectral masking
  noise_estimate = librosa.feature.spectral_estimate(S, S=None, n_fft=2048, hop_length=512)
  S_filtered = librosa.effects.reduce_noise(audio=S, estimate=noise_estimate)

  # Reconstruct audio from filtered spectrogram
  audio_filtered = librosa.istft(S_filtered)

  # Replace video audio with denoised audio
  clip.write_videofile(output_path, audio=audio_filtered, fps=clip.fps, codec="libx264")

# Example usage
video_path = "video.mov"  # Replace with your video path
remove_noise(video_path)

print(f"Video with noise removed saved to: {output_path}")


TypeError: arrays to stack must be passed as a "sequence" type such as list or tuple.